# Configure Azure ML Workspace

## Workspace parameters 

**necessary** 
1. Subscription id [Azure Portal](Azure portal)
2. resource group name [resource](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/overview#resource-groups)
3. Region of workspace [supported regions] (https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service)
4. name for workspace - if you didn't create a workspace already it will be shown on the bottom

**template for workspace parameters** 

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

## Accessing workspace 

The Azure ML Sdk will try to load the workspace from the parameters and if it doesn't work you need to re-configure your inputs 

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

## Creating a new workspace 

If you need to create a new workspace, or, if you don't have a resource group, the create workspace command will create one using the name provided 

This cell will create an Azure ML workspace provided the right parameters 

**Requirements** 
1. Permission to workspace in the resource group 
2. Permission to create a resource group if it's non-existing 
3. Not the subscription owner or contibutor and no Azure ML workspace have been created 

- If anything fails then you need to have the appropriate permissions for the required resources 

**Note** This is a basic workspace, if you want an Enterprise workspace specify the sku = 'enterprise' 

In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      sku = 'basic',
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

## Creating compute resource for training experiments 

This will create default compute clusters for use by any other future notebooks and operations 

* To create a cluster, specify computer configuration and name for cluster that's unique.

Cluster parameters: 

* **vm_size** - virtual machine type and size used in cluster. To get a list of vm sizes acailable in your region use the CLI command 

``az vm list-skus -o tsv`` 

**min_nodes** - minimum size of the cluster. IF the minimum is set to 0 then the cluster will shut down all nodes not in used. A higher number than 0 will have faster startup times but will also be billed when the cluster is not in use. 
**max_nodes** - the maximum size of the cluster. The larger a number allows for more concurrency and greater distributed processing of scale-out jobs 

> To create a CPU cluster run the cell below, the autoscale settings mean that the cluster will scaled down to 0 nodes when inactive and up to 4 nodes when busy. 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

> To create a **GPU** cluster, run the cell below. Your subscription must have sufficient quota for GPU VMs or the command will fail. 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "gpu-cluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                           min_nodes=0,
                                                           max_nodes=4)
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)